In [1]:
import sys, time, pickle
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from IPython.core.display import display, HTML
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pymongo import MongoClient
import pandas as pd
import numpy as np

from sklearn.feature_selection import mutual_info_regression

import matplotlib.pyplot as plt

# make matplotlib plot sizes larger
plt.rcParams['figure.figsize'] = [30, 20]

conf = SparkConf().setAppName('Steam Random Forest Regressor').setMaster('spark://sparkmaster:7077')
SparkContext.setSystemProperty('spark.executor.memory', '2g') # memory per executor
SparkContext.setSystemProperty('spark.executor.cores', '6') # cores per executor
SparkContext.setSystemProperty('spark.executor.instances', '3') # per worker (computer)

# https://spark.apache.org/docs/3.0.0-preview/configuration.html#dynamic-allocation
# https://stackoverflow.com/questions/26168254/how-to-set-amount-of-spark-executors
# https://blog.cloudera.com/how-to-tune-your-apache-spark-jobs-part-2/

# SparkContext.setSystemProperty("spark.shuffle.service.enabled", "True") # required for dynamic allocation below
# SparkContext.setSystemProperty("spark.dynamicAllocation.enabled", "True")
# SparkContext.setSystemProperty("spark.executor.cores", "4")
# SparkContext.setSystemProperty("spark.dynamicAllocation.minExecutors", "1")
# SparkContext.setSystemProperty("spark.dynamicAllocation.maxExecutors", "5")
# SparkContext.setSystemProperty('spark.executor.memory', '2g') # memory per executor

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [2]:
# Customizable Variables

# what percentage discount should be predicted?
percentage_discount_predict = 25

In [3]:
sc._conf.getAll()

[('spark.executor.memory', '2g'),
 ('spark.driver.host', 'jupyterlab'),
 ('spark.executor.instances', '3'),
 ('spark.app.name', 'Steam Random Forest Regressor'),
 ('spark.app.id', 'app-20210629195330-0002'),
 ('spark.executor.id', 'driver'),
 ('spark.master', 'spark://sparkmaster:7077'),
 ('spark.executor.cores', '6'),
 ('spark.app.startTime', '1625018009769'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '33139')]

In [4]:
client = MongoClient(host="192.168.1.171", port=27017)
db = client['steam']
apps = db['apps']
history = db['pricehistory']
opencritic = db['opencritic']
topgames = db['topgames']
twitch = db['twitchhistorical']

# TODO: account for cases where there are multiple publishers or developers?
ret = apps.aggregate([
    {"$match": {"updated_date": {"$exists": True}, "type": {"$in": ["game", "dlc"]}, "release_date.coming_soon": {"$ne": True}, "is_free": False}},
    {"$project": {"_id":0, "name":1, "appid":1, "release_date": "$release_date.date", "total_recommendations": "$recommendations.total",
                  "initial_price": "$price_overview.initial", "metacritic_score": "$metacritic.score", "type":1, "required_age":1,
                 "windows":"$platforms.windows", "mac":"$platforms.mac", "linux":"$platforms.linux",
                 "publisher": {"$arrayElemAt": ["$publishers", 0]}, "developer": {"$arrayElemAt": ["$developers", 0]},
                 "number_dlc": {'$size': {"$ifNull": [ "$dlc", [] ]}}, "number_genres": {'$size': {"$ifNull": [ "$genres", [] ]}}, "number_categories": {'$size': {"$ifNull": [ "$categories", [] ]}}}}
])

In [5]:
# save as pandas dataframe
df = pd.DataFrame(ret)

In [6]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,publisher,developer,number_dlc,number_genres,number_categories,total_recommendations
0,game,Legend: Hand of God,0,12630,"May 5, 2009",999.0,57.0,True,False,False,DTP Entertainment,Master Creating,0,2,1,NaN
1,game,realMyst,0,63600,"Aug 4, 2010",599.0,66.0,True,False,False,Cyan Worlds,Cyan Worlds,0,2,1,231.0
2,dlc,Crusader Kings II: African Portraits,0,210897,"Oct 16, 2012",199.0,NaN,True,True,True,Paradox Interactive,Paradox Interactive,0,1,3,NaN
3,game,Planetary Annihilation,0,233250,"Sep 5, 2014",2999.0,62.0,True,True,True,Planetary Annihilation Inc,Planetary Annihilation Inc,2,1,6,14931.0
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,"Aug 16, 2013",499.0,NaN,True,True,True,,Klei Entertainment,0,3,8,239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74975,dlc,Crossout - Wild Hunt Pack,0,812760,2018-03-15 00:00:00,NaN,NaN,True,False,False,Gaijin Distribution KFT,Targem Games,0,4,11,NaN
74976,game,Hot Brass,0,892860,2021-02-26 00:00:00,1499.0,NaN,True,True,False,Treasure Hunters FanClub,Walk with Kings,1,3,9,169.0
74977,game,mmmmm donuts arhhh......,0,1151750,2019-09-23 00:00:00,99.0,NaN,True,False,False,Game for people,Easy game,0,3,2,NaN
74978,game,Graviton,0,1140520,2019-09-16 00:00:00,199.0,NaN,True,True,False,Richter Games,Richter Games,0,1,4,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74980 entries, 0 to 74979
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type                   74980 non-null  object 
 1   name                   74980 non-null  object 
 2   required_age           74980 non-null  object 
 3   appid                  74980 non-null  int64  
 4   release_date           74980 non-null  object 
 5   initial_price          70023 non-null  float64
 6   metacritic_score       3655 non-null   float64
 7   windows                74980 non-null  bool   
 8   mac                    74980 non-null  bool   
 9   linux                  74980 non-null  bool   
 10  publisher              74980 non-null  object 
 11  developer              74150 non-null  object 
 12  number_dlc             74980 non-null  int64  
 13  number_genres          74980 non-null  int64  
 14  number_categories      74980 non-null  int64  
 15  to

In [8]:
# fix cases where it lists '17+'
df['required_age'] = df['required_age'].replace(['17+'],'17')

# convert column to datetime
df['release_date'] = pd.to_datetime(df['release_date'])

# convert column to int64 type
df['required_age'] = pd.to_numeric(df['required_age'])

In [9]:
# figure out the release day in the year
df['release_day_in_year'] = df['release_date'].dt.dayofyear

In [10]:
# aggregate the steam recommendations and tie them in

ret = apps.aggregate([
    {"$unwind":"$reviews"},
    {"$unwind":"$reviews.rollups"},
    {
        "$group": {
            "_id": "$appid",
           "sum_recommendations_up": {"$sum": "$reviews.rollups.recommendations_up"},
            "sum_recommendations_down": {"$sum": "$reviews.rollups.recommendations_down"}
        }
    }
])

# save as pandas dataframe
recommendations = pd.DataFrame(ret)
recommendations = recommendations.rename(columns={"_id": "appid"})

recommendations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53004 entries, 0 to 53003
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   appid                     53004 non-null  int64
 1   sum_recommendations_up    53004 non-null  int64
 2   sum_recommendations_down  53004 non-null  int64
dtypes: int64(3)
memory usage: 1.2 MB


In [11]:
df = pd.merge(df, recommendations, on="appid", how="left")
del recommendations

In [12]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,publisher,developer,number_dlc,number_genres,number_categories,total_recommendations,release_day_in_year,sum_recommendations_up,sum_recommendations_down
0,game,Legend: Hand of God,0,12630,2009-05-05,999.0,57.0,True,False,False,DTP Entertainment,Master Creating,0,2,1,NaN,125.0,15.0,10.0
1,game,realMyst,0,63600,2010-08-04,599.0,66.0,True,False,False,Cyan Worlds,Cyan Worlds,0,2,1,231.0,216.0,132.0,87.0
2,dlc,Crusader Kings II: African Portraits,0,210897,2012-10-16,199.0,NaN,True,True,True,Paradox Interactive,Paradox Interactive,0,1,3,NaN,290.0,19.0,7.0
3,game,Planetary Annihilation,0,233250,2014-09-05,2999.0,62.0,True,True,True,Planetary Annihilation Inc,Planetary Annihilation Inc,2,1,6,14931.0,248.0,6268.0,4576.0
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,2013-08-16,499.0,NaN,True,True,True,,Klei Entertainment,0,3,8,239.0,228.0,195.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74975,dlc,Crossout - Wild Hunt Pack,0,812760,2018-03-15,NaN,NaN,True,False,False,Gaijin Distribution KFT,Targem Games,0,4,11,NaN,74.0,NaN,NaN
74976,game,Hot Brass,0,892860,2021-02-26,1499.0,NaN,True,True,False,Treasure Hunters FanClub,Walk with Kings,1,3,9,169.0,57.0,NaN,NaN
74977,game,mmmmm donuts arhhh......,0,1151750,2019-09-23,99.0,NaN,True,False,False,Game for people,Easy game,0,3,2,NaN,266.0,NaN,NaN
74978,game,Graviton,0,1140520,2019-09-16,199.0,NaN,True,True,False,Richter Games,Richter Games,0,1,4,NaN,259.0,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74980 entries, 0 to 74979
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   type                      74980 non-null  object        
 1   name                      74980 non-null  object        
 2   required_age              74980 non-null  int64         
 3   appid                     74980 non-null  int64         
 4   release_date              74682 non-null  datetime64[ns]
 5   initial_price             70023 non-null  float64       
 6   metacritic_score          3655 non-null   float64       
 7   windows                   74980 non-null  bool          
 8   mac                       74980 non-null  bool          
 9   linux                     74980 non-null  bool          
 10  publisher                 74980 non-null  object        
 11  developer                 74150 non-null  object        
 12  number_dlc        

In [14]:
ret = history.aggregate([
    {"$match": {"discount_percent": {"$gte": percentage_discount_predict}}},
    {"$group": {"_id": "$appid", "date": {"$min": "$date"}}},
    {"$project": {"_id":0, "appid":"$_id", "date":1}}
])

In [15]:
history_df = pd.DataFrame(ret)

In [16]:
history_df

,date,appid
0,2020-12-23 03:15:38.436,1170491
1,2018-11-01 00:35:36.964,762310
2,2018-11-01 00:25:24.200,555760
3,2018-11-01 00:23:49.191,452340
4,2018-11-22 00:55:58.358,582010
...,...,...
58829,2019-10-13 10:40:33.694,921900
58830,2019-06-28 01:42:20.512,1006460
58831,2019-05-02 04:39:22.505,892240
58832,2019-05-24 00:49:45.878,1054620


In [17]:
history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58834 entries, 0 to 58833
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    58834 non-null  datetime64[ns]
 1   appid   58834 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 919.4 KB


In [18]:
df = pd.merge(df, history_df, on="appid", how="left")
del history_df

In [19]:
# find time in days between release and at least X% off
df['days_until_discount'] = df['date'] - df['release_date']
df['days_until_discount'] = df['days_until_discount'] / np.timedelta64(1,"D")

In [20]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,...,developer,number_dlc,number_genres,number_categories,total_recommendations,release_day_in_year,sum_recommendations_up,sum_recommendations_down,date,days_until_discount
0,game,Legend: Hand of God,0,12630,2009-05-05,999.0,57.0,True,False,False,...,Master Creating,0,2,1,NaN,125.0,15.0,10.0,NaT,NaN
1,game,realMyst,0,63600,2010-08-04,599.0,66.0,True,False,False,...,Cyan Worlds,0,2,1,231.0,216.0,132.0,87.0,NaT,NaN
2,dlc,Crusader Kings II: African Portraits,0,210897,2012-10-16,199.0,NaN,True,True,True,...,Paradox Interactive,0,1,3,NaN,290.0,19.0,7.0,2018-09-07 02:06:54.097,2152.088126
3,game,Planetary Annihilation,0,233250,2014-09-05,2999.0,62.0,True,True,True,...,Planetary Annihilation Inc,2,1,6,14931.0,248.0,6268.0,4576.0,NaT,NaN
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,2013-08-16,499.0,NaN,True,True,True,...,Klei Entertainment,0,3,8,239.0,228.0,195.0,19.0,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74975,dlc,Crossout - Wild Hunt Pack,0,812760,2018-03-15,NaN,NaN,True,False,False,...,Targem Games,0,4,11,NaN,74.0,NaN,NaN,2018-12-21 19:21:33.018,281.806632
74976,game,Hot Brass,0,892860,2021-02-26,1499.0,NaN,True,True,False,...,Walk with Kings,1,3,9,169.0,57.0,NaN,NaN,NaT,NaN
74977,game,mmmmm donuts arhhh......,0,1151750,2019-09-23,99.0,NaN,True,False,False,...,Easy game,0,3,2,NaN,266.0,NaN,NaN,2019-10-30 13:22:08.125,37.557038
74978,game,Graviton,0,1140520,2019-09-16,199.0,NaN,True,True,False,...,Richter Games,0,1,4,NaN,259.0,NaN,NaN,2019-12-20 23:59:57.086,95.999966


In [21]:
# get top games information

ret = topgames.aggregate([
    {"$match": {}},
    {"$project": {"_id":0, "date":1, "appid":1, "currentplayers":1, "peaktoday":1}}
])
tgames = pd.DataFrame(ret)
tgames['appid'] = pd.to_numeric(tgames['appid'])

tgames = pd.merge(tgames, df[["appid", "release_date"]], on="appid", how="left").dropna()

tgames

,date,appid,currentplayers,peaktoday,release_date
0,2019-01-08 04:04:02.491,578080,301313,970440,2017-12-21
3,2019-01-08 04:04:02.491,359550,63224,101320,2015-12-01
6,2019-01-08 04:04:02.491,271590,42480,90011,2015-04-13
7,2019-01-08 04:04:02.491,252490,39405,66405,2018-02-08
8,2019-01-08 04:04:02.491,582010,38731,74605,2018-08-09
...,...,...,...,...,...
7145166,2021-06-30 01:41:34.004,646570,7415,10166,2019-01-23
7145167,2021-06-30 01:41:34.004,1097150,7365,13390,2020-08-03
7145168,2021-06-30 01:41:34.004,552520,7355,11372,2018-03-26
7145169,2021-06-30 01:41:34.004,270880,7030,7112,2016-02-02


In [22]:
tgames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5419002 entries, 0 to 7145170
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   appid           int64         
 2   currentplayers  int64         
 3   peaktoday       int64         
 4   release_date    datetime64[ns]
dtypes: datetime64[ns](2), int64(3)
memory usage: 248.1 MB


In [23]:
# find difference between entry date and release date
tgames['days_since_release'] = (tgames['date'] - tgames['release_date']).dt.days

#tgames.query("appid==1097150 & days_since_release>=0 & days_since_release<=30")
tgames

,date,appid,currentplayers,peaktoday,release_date,days_since_release
0,2019-01-08 04:04:02.491,578080,301313,970440,2017-12-21,383
3,2019-01-08 04:04:02.491,359550,63224,101320,2015-12-01,1134
6,2019-01-08 04:04:02.491,271590,42480,90011,2015-04-13,1366
7,2019-01-08 04:04:02.491,252490,39405,66405,2018-02-08,334
8,2019-01-08 04:04:02.491,582010,38731,74605,2018-08-09,152
...,...,...,...,...,...,...
7145166,2021-06-30 01:41:34.004,646570,7415,10166,2019-01-23,889
7145167,2021-06-30 01:41:34.004,1097150,7365,13390,2020-08-03,331
7145168,2021-06-30 01:41:34.004,552520,7355,11372,2018-03-26,1192
7145169,2021-06-30 01:41:34.004,270880,7030,7112,2016-02-02,1975


In [24]:
bins = range(-1, 7, 1)
groups = tgames.groupby(['appid', pd.cut(tgames.days_since_release, bins, labels=["start_players_{:01d}".format(x) for x in bins[:-1]])])
groups = groups.mean().currentplayers.unstack()
groups = groups.fillna(0)

#groups[groups.index.isin([1097150])]
groups

days_since_release,start_players_-1,start_players_0,start_players_1,start_players_2,start_players_3,start_players_4,start_players_5
appid,,,,,,,
10,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000
70,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000
220,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000
240,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000
550,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000
...,...,...,...,...,...,...,...
1478540,0.0,0.0,0.0,0.000,25521.62069,21264.228916,19905.925
1526490,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000
1530870,0.0,0.0,0.0,0.000,0.00000,0.000000,0.000


In [25]:
df = pd.merge(df, groups, left_on="appid", right_index=True, how="left")
del groups
del tgames

df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,...,sum_recommendations_down,date,days_until_discount,start_players_-1,start_players_0,start_players_1,start_players_2,start_players_3,start_players_4,start_players_5
0,game,Legend: Hand of God,0,12630,2009-05-05,999.0,57.0,True,False,False,...,10.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,realMyst,0,63600,2010-08-04,599.0,66.0,True,False,False,...,87.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dlc,Crusader Kings II: African Portraits,0,210897,2012-10-16,199.0,NaN,True,True,True,...,7.0,2018-09-07 02:06:54.097,2152.088126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,Planetary Annihilation,0,233250,2014-09-05,2999.0,62.0,True,True,True,...,4576.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,2013-08-16,499.0,NaN,True,True,True,...,19.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74975,dlc,Crossout - Wild Hunt Pack,0,812760,2018-03-15,NaN,NaN,True,False,False,...,NaN,2018-12-21 19:21:33.018,281.806632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74976,game,Hot Brass,0,892860,2021-02-26,1499.0,NaN,True,True,False,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74977,game,mmmmm donuts arhhh......,0,1151750,2019-09-23,99.0,NaN,True,False,False,...,NaN,2019-10-30 13:22:08.125,37.557038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74978,game,Graviton,0,1140520,2019-09-16,199.0,NaN,True,True,False,...,NaN,2019-12-20 23:59:57.086,95.999966,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74980 entries, 0 to 74979
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   type                      74980 non-null  object        
 1   name                      74980 non-null  object        
 2   required_age              74980 non-null  int64         
 3   appid                     74980 non-null  int64         
 4   release_date              74682 non-null  datetime64[ns]
 5   initial_price             70023 non-null  float64       
 6   metacritic_score          3655 non-null   float64       
 7   windows                   74980 non-null  bool          
 8   mac                       74980 non-null  bool          
 9   linux                     74980 non-null  bool          
 10  publisher                 74980 non-null  object        
 11  developer                 74150 non-null  object        
 12  number_dlc        

In [27]:
# get twitch games

ret = twitch.aggregate([
    {"$match": {}},
    {"$project": {"_id":0, "date":1, "name":1, "gamerank":1, "viewer_count":1}}
])
twitch_df = pd.DataFrame(ret)

twitch_df = pd.merge(twitch_df, df[["name", "release_date"]], on="name", how="left").dropna()

twitch_df

,viewer_count,date,name,gamerank,release_date
300,9084,2019-07-14 19:01:26.131,Grand Theft Auto V,4,2015-04-13
301,8128,2019-07-14 19:01:26.144,Grand Theft Auto V,4,2015-04-13
302,7858,2019-07-14 19:01:26.157,Grand Theft Auto V,4,2015-04-13
303,5847,2019-07-14 19:01:26.170,Grand Theft Auto V,4,2015-04-13
304,4803,2019-07-14 19:01:26.183,Grand Theft Auto V,4,2015-04-13
...,...,...,...,...,...
13345933,5,2021-06-29 23:33:06.232,Fall Guys: Ultimate Knockout,98,2020-08-03
13345934,5,2021-06-29 23:33:06.234,Fall Guys: Ultimate Knockout,98,2020-08-03
13345935,5,2021-06-29 23:33:06.237,Fall Guys: Ultimate Knockout,98,2020-08-03
13345936,5,2021-06-29 23:33:06.239,Fall Guys: Ultimate Knockout,98,2020-08-03


In [28]:
twitch_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3932997 entries, 300 to 13345937
Data columns (total 5 columns):
 #   Column        Dtype         
---  ------        -----         
 0   viewer_count  int64         
 1   date          datetime64[ns]
 2   name          object        
 3   gamerank      int64         
 4   release_date  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 180.0+ MB


In [29]:
# find difference between entry date and release date
twitch_df['days_since_release'] = (twitch_df['date'] - twitch_df['release_date']).dt.days

#twitch_df.query("appid==1097150 & days_since_release>=0 & days_since_release<=30")
twitch_df

,viewer_count,date,name,gamerank,release_date,days_since_release
300,9084,2019-07-14 19:01:26.131,Grand Theft Auto V,4,2015-04-13,1553
301,8128,2019-07-14 19:01:26.144,Grand Theft Auto V,4,2015-04-13,1553
302,7858,2019-07-14 19:01:26.157,Grand Theft Auto V,4,2015-04-13,1553
303,5847,2019-07-14 19:01:26.170,Grand Theft Auto V,4,2015-04-13,1553
304,4803,2019-07-14 19:01:26.183,Grand Theft Auto V,4,2015-04-13,1553
...,...,...,...,...,...,...
13345933,5,2021-06-29 23:33:06.232,Fall Guys: Ultimate Knockout,98,2020-08-03,330
13345934,5,2021-06-29 23:33:06.234,Fall Guys: Ultimate Knockout,98,2020-08-03,330
13345935,5,2021-06-29 23:33:06.237,Fall Guys: Ultimate Knockout,98,2020-08-03,330
13345936,5,2021-06-29 23:33:06.239,Fall Guys: Ultimate Knockout,98,2020-08-03,330


In [30]:
bins = range(-1, 7, 1)
groups_viewers = twitch_df.groupby(['name', pd.cut(twitch_df.days_since_release, bins, labels=["start_twitch_viewers_{:01d}".format(x) for x in bins[:-1]])])
groups_viewers = groups_viewers.mean().viewer_count.unstack()
groups_viewers = groups_viewers.fillna(0)

groups_viewers

days_since_release,start_twitch_viewers_-1,start_twitch_viewers_0,start_twitch_viewers_1,start_twitch_viewers_2,start_twitch_viewers_3,start_twitch_viewers_4,start_twitch_viewers_5
name,,,,,,,
.hack//G.U. Last Recode,0.0,0.0,0.00,0.0,0.0,0.0,0.0
112 Operator,0.0,1721.0,0.00,0.0,0.0,0.0,0.0
12 Labours of Hercules II: The Cretan Bull,0.0,0.0,0.00,0.0,0.0,0.0,0.0
140,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1406,0.0,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
iRacing,0.0,0.0,0.00,0.0,0.0,0.0,0.0
ibb & obb,0.0,0.0,0.00,0.0,0.0,0.0,0.0
looK INside,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [31]:
bins = range(-1, 7, 1)
groups_rank = twitch_df.groupby(['name', pd.cut(twitch_df.days_since_release, bins, labels=["start_twitch_rank_{:01d}".format(x) for x in bins[:-1]])])
groups_rank = groups_rank.mean().gamerank.unstack()
groups_rank = groups_rank.fillna(0)

groups_rank

days_since_release,start_twitch_rank_-1,start_twitch_rank_0,start_twitch_rank_1,start_twitch_rank_2,start_twitch_rank_3,start_twitch_rank_4,start_twitch_rank_5
name,,,,,,,
.hack//G.U. Last Recode,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112 Operator,0.0,23.0,0.0,0.0,0.0,0.0,0.0
12 Labours of Hercules II: The Cretan Bull,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1406,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
iRacing,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ibb & obb,0.0,0.0,0.0,0.0,0.0,0.0,0.0
looK INside,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# merge twitch viewership and gamerank back into master df
df = pd.merge(df, groups_viewers, left_on="name", right_index=True, how="left")
del groups_viewers
df = pd.merge(df, groups_rank, left_on="name", right_index=True, how="left")
del groups_rank

del twitch_df

In [33]:
# fill in all the NaN spots
players_filter_col = [c for c in df.columns if c.startswith('start')]
print(players_filter_col)
df[players_filter_col] = df[players_filter_col].fillna(0)

['start_players_-1', 'start_players_0', 'start_players_1', 'start_players_2', 'start_players_3', 'start_players_4', 'start_players_5', 'start_twitch_viewers_-1', 'start_twitch_viewers_0', 'start_twitch_viewers_1', 'start_twitch_viewers_2', 'start_twitch_viewers_3', 'start_twitch_viewers_4', 'start_twitch_viewers_5', 'start_twitch_rank_-1', 'start_twitch_rank_0', 'start_twitch_rank_1', 'start_twitch_rank_2', 'start_twitch_rank_3', 'start_twitch_rank_4', 'start_twitch_rank_5']


In [34]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,...,start_twitch_viewers_3,start_twitch_viewers_4,start_twitch_viewers_5,start_twitch_rank_-1,start_twitch_rank_0,start_twitch_rank_1,start_twitch_rank_2,start_twitch_rank_3,start_twitch_rank_4,start_twitch_rank_5
0,game,Legend: Hand of God,0,12630,2009-05-05,999.0,57.0,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,game,realMyst,0,63600,2010-08-04,599.0,66.0,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dlc,Crusader Kings II: African Portraits,0,210897,2012-10-16,199.0,NaN,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,game,Planetary Annihilation,0,233250,2014-09-05,2999.0,62.0,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,2013-08-16,499.0,NaN,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74975,dlc,Crossout - Wild Hunt Pack,0,812760,2018-03-15,NaN,NaN,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74976,game,Hot Brass,0,892860,2021-02-26,1499.0,NaN,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74977,game,mmmmm donuts arhhh......,0,1151750,2019-09-23,99.0,NaN,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74978,game,Graviton,0,1140520,2019-09-16,199.0,NaN,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74980 entries, 0 to 74979
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   type                      74980 non-null  object        
 1   name                      74980 non-null  object        
 2   required_age              74980 non-null  int64         
 3   appid                     74980 non-null  int64         
 4   release_date              74682 non-null  datetime64[ns]
 5   initial_price             70023 non-null  float64       
 6   metacritic_score          3655 non-null   float64       
 7   windows                   74980 non-null  bool          
 8   mac                       74980 non-null  bool          
 9   linux                     74980 non-null  bool          
 10  publisher                 74980 non-null  object        
 11  developer                 74150 non-null  object        
 12  number_dlc        

In [36]:
ret = opencritic.aggregate([
    {"$match": {"steamId": {"$exists": True, "$nin": [None, "", "null", "NaN"], "$not": {"$type": 10}}}},
    {"$project": {"_id":0, "appid":"$steamId", "game_type":"$type", "hasLootBoxes":1, "isMajorTitle":1, "medianScore":1, "numReviews":1,
                  "numTopCriticReviews":1, "percentRecommended":1, "percentile":1, "tier":1, "topCriticScore":1,
                 "number_platforms": {'$size': {"$ifNull": [ "$Platforms", [] ]}}, "number_skus": {'$size': {"$ifNull": [ "$Skus", [] ]}}}}
])

In [37]:
opencritic_df = pd.DataFrame(ret)

# fix cases where we don't have data
opencritic_df = opencritic_df.replace({'medianScore': -1, 'percentRecommended': -1, 'percentile': -1, 'topCriticScore': -1}, np.nan)

opencritic_df

In [38]:
opencritic_df.describe()

In [39]:
# convert object to int64 type
opencritic_df["appid"] = pd.to_numeric(opencritic_df["appid"])

In [40]:
# NaNs will be pruned out later, in order to use as much of the sparse OpenCritic data that we have, we fill it in with zeros
opencritic_df.fillna(0, inplace=True)

In [41]:
opencritic_df

In [42]:
opencritic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4791 entries, 0 to 4790
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hasLootBoxes         4791 non-null   object 
 1   isMajorTitle         4791 non-null   bool   
 2   medianScore          4791 non-null   float64
 3   numReviews           4791 non-null   int64  
 4   numTopCriticReviews  4791 non-null   int64  
 5   percentRecommended   4791 non-null   float64
 6   percentile           4791 non-null   float64
 7   tier                 4791 non-null   object 
 8   topCriticScore       4791 non-null   float64
 9   appid                4791 non-null   int64  
 10  game_type            4791 non-null   object 
 11  number_platforms     4791 non-null   int64  
 12  number_skus          4791 non-null   int64  
dtypes: bool(1), float64(4), int64(5), object(3)
memory usage: 454.0+ KB


In [43]:
df = pd.merge(df, opencritic_df, on="appid", how="left")
del opencritic_df

In [44]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,...,medianScore,numReviews,numTopCriticReviews,percentRecommended,percentile,tier,topCriticScore,game_type,number_platforms,number_skus
0,game,Legend: Hand of God,0,12630,2009-05-05,999.0,57.0,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,realMyst,0,63600,2010-08-04,599.0,66.0,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dlc,Crusader Kings II: African Portraits,0,210897,2012-10-16,199.0,NaN,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,Planetary Annihilation,0,233250,2014-09-05,2999.0,62.0,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dlc,Mark of the Ninja: Special Edition DLC,17,239570,2013-08-16,499.0,NaN,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75061,dlc,Crossout - Wild Hunt Pack,0,812760,2018-03-15,NaN,NaN,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75062,game,Hot Brass,0,892860,2021-02-26,1499.0,NaN,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75063,game,mmmmm donuts arhhh......,0,1151750,2019-09-23,99.0,NaN,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75064,game,Graviton,0,1140520,2019-09-16,199.0,NaN,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# https://analyticsindiamag.com/5-ways-handle-missing-values-machine-learning-datasets/
# TODO: use mean or median for cases where we have missing data



# just assume anything not filled in doesn't have loot boxes
#df = df.replace({'hasLootBoxes': np.nan}, False)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75066 entries, 0 to 75065
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   type                      75066 non-null  object        
 1   name                      75066 non-null  object        
 2   required_age              75066 non-null  int64         
 3   appid                     75066 non-null  int64         
 4   release_date              74768 non-null  datetime64[ns]
 5   initial_price             70105 non-null  float64       
 6   metacritic_score          3696 non-null   float64       
 7   windows                   75066 non-null  bool          
 8   mac                       75066 non-null  bool          
 9   linux                     75066 non-null  bool          
 10  publisher                 75066 non-null  object        
 11  developer                 74236 non-null  object        
 12  number_dlc        

In [47]:
# we can't run prediction with empty values
# make a copy so we can continue modifying the dataframe
df = df.dropna().copy()

In [48]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,...,medianScore,numReviews,numTopCriticReviews,percentRecommended,percentile,tier,topCriticScore,game_type,number_platforms,number_skus
329,game,OVERKILL's The Walking Dead,17,717690,2018-11-06,2999.0,51.0,True,False,False,...,55.0,39.0,28.0,10.526316,9.0,Weak,53.178571,BASE,5.0,10.0
3495,game,Inside My Radio,0,357720,2015-05-11,1499.0,68.0,True,False,False,...,60.0,14.0,11.0,21.428571,24.0,Weak,63.181818,BASE,6.0,9.0
3497,game,Richard & Alice,0,279260,2014-06-05,599.0,70.0,True,False,False,...,80.0,10.0,9.0,60.000000,52.0,Fair,72.222222,BASE,1.0,3.0
3534,dlc,Warhammer: Vermintide 2 - Winds of Magic,0,1033060,2019-08-13,1999.0,67.0,True,False,False,...,69.0,6.0,4.0,16.666667,20.0,Weak,61.250000,BASE,1.0,5.0
3617,game,SKYHILL,0,382140,2015-10-06,1499.0,63.0,True,True,True,...,65.0,19.0,17.0,16.666667,28.0,Weak,65.352941,BASE,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74580,dlc,Cities: Skylines - Snowfall,0,420610,2016-02-18,1299.0,72.0,True,True,True,...,75.0,16.0,13.0,38.461538,52.0,Fair,72.923077,BASE,1.0,7.0
74614,game,Pro Cycling Manager 2016,0,408760,2016-06-16,999.0,64.0,True,False,False,...,0.0,4.0,2.0,0.000000,0.0,,0.000000,BASE,5.0,5.0
74946,game,OmniBus,0,415740,2016-05-26,499.0,55.0,True,True,True,...,65.0,5.0,5.0,20.000000,21.0,Weak,62.400000,BASE,1.0,6.0
75058,game,Shadow Warrior 2,17,324800,2016-10-13,3999.0,78.0,True,False,False,...,80.0,80.0,56.0,61.538462,66.0,Strong,76.839286,BASE,5.0,7.0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1354 entries, 329 to 75060
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   type                      1354 non-null   object        
 1   name                      1354 non-null   object        
 2   required_age              1354 non-null   int64         
 3   appid                     1354 non-null   int64         
 4   release_date              1354 non-null   datetime64[ns]
 5   initial_price             1354 non-null   float64       
 6   metacritic_score          1354 non-null   float64       
 7   windows                   1354 non-null   bool          
 8   mac                       1354 non-null   bool          
 9   linux                     1354 non-null   bool          
 10  publisher                 1354 non-null   object        
 11  developer                 1354 non-null   object        
 12  number_dlc       

In [50]:
feature_list = ['required_age', 'metacritic_score', 'sum_recommendations_up', 'sum_recommendations_down', 'total_recommendations', 'initial_price', 'windows_num',
                'mac_num', 'linux_num', 'developer_num', 'publisher_num',
               'number_dlc', 'release_day_in_year', 'number_genres', 'number_categories']

# items from OpenCritic
feature_list = feature_list + ['type_num', 'hasLootBoxes_num', 'isMajorTitle_num',
            'tier_num', 'game_type_num', 'medianScore', 'numReviews', 'numTopCriticReviews', 'percentRecommended', 'percentile',
            'topCriticScore', 'number_platforms', 'number_skus']

feature_list = feature_list + players_filter_col
print(feature_list)

assembler = VectorAssembler(inputCols=feature_list, outputCol="features")

['required_age', 'metacritic_score', 'sum_recommendations_up', 'sum_recommendations_down', 'total_recommendations', 'initial_price', 'windows_num', 'mac_num', 'linux_num', 'developer_num', 'publisher_num', 'number_dlc', 'release_day_in_year', 'number_genres', 'number_categories', 'type_num', 'hasLootBoxes_num', 'isMajorTitle_num', 'tier_num', 'game_type_num', 'medianScore', 'numReviews', 'numTopCriticReviews', 'percentRecommended', 'percentile', 'topCriticScore', 'number_platforms', 'number_skus', 'start_players_-1', 'start_players_0', 'start_players_1', 'start_players_2', 'start_players_3', 'start_players_4', 'start_players_5', 'start_twitch_viewers_-1', 'start_twitch_viewers_0', 'start_twitch_viewers_1', 'start_twitch_viewers_2', 'start_twitch_viewers_3', 'start_twitch_viewers_4', 'start_twitch_viewers_5', 'start_twitch_rank_-1', 'start_twitch_rank_0', 'start_twitch_rank_1', 'start_twitch_rank_2', 'start_twitch_rank_3', 'start_twitch_rank_4', 'start_twitch_rank_5']


In [51]:
# https://www.silect.is/blog/2019/4/2/random-forest-in-spark-ml
# an example Random Forest in Spark

In [52]:
# encode columns to numeric so we can run regression(s)

# items from OpenCritic
df['type_num'] = pd.factorize(df['type'])[0]
df['hasLootBoxes_num'] = pd.factorize(df['hasLootBoxes'])[0]
df['isMajorTitle_num'] = pd.factorize(df['isMajorTitle'])[0]
df['tier_num'] = pd.factorize(df['tier'])[0]
df['game_type_num'] = pd.factorize(df['game_type'])[0]

df['windows_num'] = pd.factorize(df['windows'])[0]
df['mac_num'] = pd.factorize(df['mac'])[0]
df['linux_num'] = pd.factorize(df['linux'])[0]
df['developer_num'] = pd.factorize(df['developer'])[0]
df['publisher_num'] = pd.factorize(df['publisher'])[0]

In [53]:
df

,type,name,required_age,appid,release_date,initial_price,metacritic_score,windows,mac,linux,...,type_num,hasLootBoxes_num,isMajorTitle_num,tier_num,game_type_num,windows_num,mac_num,linux_num,developer_num,publisher_num
329,game,OVERKILL's The Walking Dead,17,717690,2018-11-06,2999.0,51.0,True,False,False,...,0,0,0,0,0,0,0,0,0,0
3495,game,Inside My Radio,0,357720,2015-05-11,1499.0,68.0,True,False,False,...,0,0,0,0,0,0,0,0,1,1
3497,game,Richard & Alice,0,279260,2014-06-05,599.0,70.0,True,False,False,...,0,0,0,1,0,0,0,0,2,2
3534,dlc,Warhammer: Vermintide 2 - Winds of Magic,0,1033060,2019-08-13,1999.0,67.0,True,False,False,...,1,0,0,0,0,0,0,0,3,3
3617,game,SKYHILL,0,382140,2015-10-06,1499.0,63.0,True,True,True,...,0,0,0,0,0,0,1,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74580,dlc,Cities: Skylines - Snowfall,0,420610,2016-02-18,1299.0,72.0,True,True,True,...,1,0,0,1,0,0,1,1,108,60
74614,game,Pro Cycling Manager 2016,0,408760,2016-06-16,999.0,64.0,True,False,False,...,0,0,0,3,0,0,0,0,67,49
74946,game,OmniBus,0,415740,2016-05-26,499.0,55.0,True,True,True,...,0,0,0,0,0,0,1,1,970,47
75058,game,Shadow Warrior 2,17,324800,2016-10-13,3999.0,78.0,True,False,False,...,0,0,1,4,0,0,0,0,654,47


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1354 entries, 329 to 75060
Data columns (total 64 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   type                      1354 non-null   object        
 1   name                      1354 non-null   object        
 2   required_age              1354 non-null   int64         
 3   appid                     1354 non-null   int64         
 4   release_date              1354 non-null   datetime64[ns]
 5   initial_price             1354 non-null   float64       
 6   metacritic_score          1354 non-null   float64       
 7   windows                   1354 non-null   bool          
 8   mac                       1354 non-null   bool          
 9   linux                     1354 non-null   bool          
 10  publisher                 1354 non-null   object        
 11  developer                 1354 non-null   object        
 12  number_dlc       

In [54]:
# TODO: qq plot, residuals, etc.
# https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Regression/Regression_Diagnostics.ipynb
# https://towardsdatascience.com/how-do-you-check-the-quality-of-your-regression-model-in-python-fa61759ff685


In [55]:
# perform k-means clustering to create an additional numeric column for feature engineering
# https://www.kaggle.com/ryanholbrook/clustering-with-k-means
# https://spark.apache.org/docs/latest/ml-clustering.html

# convert pandas dataframe to spark dataframe
temp_df = sqlContext.createDataFrame(df)

dataset = assembler.transform(temp_df)
dataset.select("features").show(truncate=False)

cost = np.ones(17)
for k in range(2,17):
    # Trains a k-means model.
    kmeans = KMeans(k=k).setSeed(1)
    model = kmeans.fit(dataset)
    
    # Make predictions
    predictions = model.transform(dataset)

    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()

    silhouette = evaluator.evaluate(predictions)
    print("k: " + str(k) + " - Silhouette with squared euclidean distance = " + str(silhouette))
    
    cost[k] = silhouette

    # Shows the result.
    centers = model.clusterCenters()
    #print("Cluster Centers: ")
    #for center in centers:
        #print(center)
    
del temp_df

TypeError: field hasLootBoxes: Can not merge type <class 'pyspark.sql.types.BooleanType'> and <class 'pyspark.sql.types.LongType'>

In [ ]:
plt.plot(np.arange(1,18), cost, linestyle='solid')
plt.ylabel('Silhouette cost')
plt.xlabel('k clusters')
plt.title('Cost from changing k clusters')

In [ ]:
# Trains a k-means model.
kmeans = KMeans(k=10).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

cost[k] = silhouette

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)
    
cluster_predictions = predictions.toPandas()

In [ ]:
cluster_predictions

In [ ]:
cluster_predictions['prediction'].unique()

In [ ]:
df['cluster_prediction'] = cluster_predictions['prediction'].values

In [ ]:
df['cluster_prediction'].unique()

In [ ]:
feature_list = feature_list + ['cluster_prediction']
print(feature_list)

assembler = VectorAssembler(inputCols=feature_list, outputCol="features")

In [ ]:
# persist and serialize objects to disk for next step
pickle.dump(df, open("/work/df.p", "wb"))
pickle.dump(feature_list, open("/work/feature_list.p", "wb"))
pickle.dump(assembler, open("/work/assembler.p", "wb"))

In [ ]:
sc.stop()